# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [12]:
from openai import OpenAI

client = OpenAI(
    # 中转Key或者直连Key
    api_key='sk-Y37XZ0XBQkz9gLma43C2D35679Ce4615910d500169C93d23',         
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1' 
)  # 初始化 OpenAI 客户端

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图描绘了一个宁静且自然美丽的风景。图片中，一条木制步道穿过一片绿色的草地，引导观者的视线向远处延伸。草地两侧是郁郁葱葱的植被和树木，远处的树木轮廓在天际线上形成一道美丽的边框。\n\n步道两边的草地呈现出鲜明的绿色，草丛中可能生活着各种小动物和昆虫，是自然生态系统的一部分。天空呈现出淡蓝色，布满了淡淡的白云，增加了画面的深度和广阔感。\n\n整体来说，这幅图片展示了一种平和而诱人的自然景象，是很多人在日常忙碌生活中向往的宁静与自然之美。这样的场景也可能是户外散步和放松的好地方，让人', role='assistant', function_call=None, tool_calls=None))


In [13]:
response.choices[0].message.content

'这幅图描绘了一个宁静且自然美丽的风景。图片中，一条木制步道穿过一片绿色的草地，引导观者的视线向远处延伸。草地两侧是郁郁葱葱的植被和树木，远处的树木轮廓在天际线上形成一道美丽的边框。\n\n步道两边的草地呈现出鲜明的绿色，草丛中可能生活着各种小动物和昆虫，是自然生态系统的一部分。天空呈现出淡蓝色，布满了淡淡的白云，增加了画面的深度和广阔感。\n\n整体来说，这幅图片展示了一种平和而诱人的自然景象，是很多人在日常忙碌生活中向往的宁静与自然之美。这样的场景也可能是户外散步和放松的好地方，让人'

### 封装成一个函数 query_image_description

In [14]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(
    # 中转Key或者直连Key
    api_key='sk-Y37XZ0XBQkz9gLma43C2D35679Ce4615910d500169C93d23',         
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1' 
)  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [15]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这是一幅幽默图片，展示了两种不同形象的柴犬。左侧的柴犬经过了“16岁的挺”处理，被描绘成拥有健壮的身体和肌肉，形象魁梧，看起来非常强壮和有力。右侧的柴犬则是“工作后的状”被描绘成坐着，体型略显圆润、看起来较为悠闲和满足。这幅图通过夸张的手法，表达出年轻时的朝气和工作后的放松颓态，含有幽默与讽刺的元素。文字部分则强调了不同生活方式和态度的对比，具有一定的娱乐和讽刺意味。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [18]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(
    # 中转Key或者直连Key
    api_key='sk-Y37XZ0XBQkz9gLma43C2D35679Ce4615910d500169C93d23',         
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1' 
)  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [19]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图表显示了1980年至2020年间美国、中国、日本和德国的国内生产总值（GDP）比较。各国GDP以万亿美元计算。

从图中可以看出：
1. **美国（蓝色线）**：美国的GDP在这40年间持续增长，从1980年的不到5万亿美元增长到2020年接近20万亿美元。

2. **中国（红色线）**：1980年时中国的GDP相对较低，但从1990年代开始迅速增长，尤其是2000年后增长更为显著，至2020年接近15万亿美元。

3. **日本（紫色线）**：日本的GDP在1980年代和1990年代初期迅速增长，达到高峰后在1990年代末到2000年初期略有波动下降，之后趋于稳定在5万亿美元左右。

4. **德国（绿色线）**：德国的GDP增长相对平稳，从1980年的不到1万亿美元缓慢增加，到2020年约为4万亿美元。

总体而言，这幅图表展示了四个主要经济体在过去几十年的经济增长情况，尤其突出了中国的快速增长和美国的稳定领先。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [27]:
content = query_base64_image_description("./images/note.jpg")
print(content)

此图显示的是一张手写的英语语法练习纸，主要内容包含英语中表示未来时的不同表达方式和结构。在纸上列出了六种不同的结构来表示未来行为或状态：

1. 使用“be going to”结构，如 “I'm meeting Jack at the station at 4 tomorrow afternoon” 和 “I think I am going to faint.”
2. 使用“be to do”结构，如 “The shop is to be opened in May” 和 “Am I to take over his work?”
3. 使用“be about to do”结构，如 “He is about to leave” 和 “The movie is about to begin.”
4. 使用简单的动词形式来询问关于未来的计划，如 "When does the show begin?" 和 "The ship leaves for Shanghai in two hours."
5. 使用“be on the point (verge, brink) of”结构来表示即将发生的动作，如 “He is on the point of going on a trip.”
6. 使用“will be doing”结构，如 “He will soon be working for the company.”

这些练习有助于学习者了解和练习英语中表示未来计划和行为的不同语法结构。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [28]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

此图显示的是一张手写的英语语法练习纸，主要内容包含英语中表示未来时的不同表达方式和结构。在纸上列出了六种不同的结构来表示未来行为或状态：

1. 使用“be going to”结构，如 “I'm meeting Jack at the station at 4 tomorrow afternoon” 和 “I think I am going to faint.”
2. 使用“be to do”结构，如 “The shop is to be opened in May” 和 “Am I to take over his work?”
3. 使用“be about to do”结构，如 “He is about to leave” 和 “The movie is about to begin.”
4. 使用简单的动词形式来询问关于未来的计划，如 "When does the show begin?" 和 "The ship leaves for Shanghai in two hours."
5. 使用“be on the point (verge, brink) of”结构来表示即将发生的动作，如 “He is on the point of going on a trip.”
6. 使用“will be doing”结构，如 “He will soon be working for the company.”

这些练习有助于学习者了解和练习英语中表示未来计划和行为的不同语法结构。

![](./images/handwriting_1.jpg)

In [ ]:
content = query_base64_image_description("./images/note.jpg")
display(Markdown(content))

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。